In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from copy import copy, deepcopy
from tqdm import tqdm
from pandarallel import pandarallel
import itertools
import time
import jieba
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import os

In [8]:
skill_data_dir = 'data/task3/data'
data_dir = 'data'
out_dir = 'out'
version = 'games'
sep = '\001'
tqdm.pandas()

### 公共函数

In [5]:
def is_chinese(uchar):
    """判断一个unicode是否是汉字"""
    if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
        return True
    else:
        return False


def inc_chinese_char(word):
    for c in word:
        if is_chinese(c):
            return True
    return False


def count_skill_words(d):
    k_count = 0
    v_count = 0
    for k, v in zip(d['1th_tag'], d['words']):
        k_count += 1
        v_count += len(v)
    print(k_count)
    print(v_count)


def mkdir(path):
    folder = os.path.exists(path)
    if not folder:   # 判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)   # makedirs 创建文件时如果路径不存在会创建这个路径
        print('Creating new folder...')
        print('Created!')
    else:
        print('This folder is already exist')

        
def clean_split_sentence(sentence):
    '''
    清理并切分句子
    :param sentence: 句子
    :return: 切分后的短句的list
    '''
    # 将所有的标点都转化为','
    sentences = sentence.replace('，', ',').replace('。', ',').replace(';', ',').replace('.', ',').replace(' ', ',').split(',')
    res = []
    for sent in sentences:
        sent = sent.strip()
        if sent != '':
            res.append(sent)
    return res


def jieba_lcut(jd):
    '''
    对一个JD进行结巴分词。
    :param jd: JD包含的短句list
    :return: 分词后的tokens
    '''
    res = []
    for sent in jd:
        res.extend(jieba.lcut(sent))
    return res


def jieba_jd_lcut(jd):
    '''
    对raw JD进行结巴分词。
    :param jd: 原始JD
    :return: 分词后的tokens
    '''
    jd = clean_split_sentence(jd)
    return jieba_lcut(jd)

### Stopwords

In [6]:
with open('data/task3/data/stopwords/cn_stopwords.txt', encoding='utf8') as f:
    cn_stopwords = [token.strip() for token in f.readlines() if token.strip() != '']
with open('data/task3/data/stopwords/hit_stopwords.txt', encoding='utf8') as f:
    hit_stopwords = [token.strip() for token in f.readlines() if token.strip() != '']
with open('data/task3/data/stopwords/baidu_stopwords.txt', encoding='utf8') as f:
    baidu_stopwords = [token.strip() for token in f.readlines() if token.strip() != '']
with open('data/task3/data/stopwords/scu_stopwords.txt', encoding='utf8') as f:
    scu_stopwords = [token.strip() for token in f.readlines() if token.strip() != '']
stopwords = set(cn_stopwords + hit_stopwords)

### 载入词典

In [9]:
d = pd.read_parquet(os.path.join(skill_data_dir, 'skill_dict_reduce_v2.parquet'))

In [10]:
d

,1th_tag,words
0,系统优化,"[规划设计优化, 规则流程优化, 视觉优化, 视觉设计优化, 视频优化, 视频延迟优化, 视..."
1,广告营销,"[视频广告, 视频广告创意, 视频广告制作, 视频广告剪辑, 视频广告审核, 视频广告投放,..."
2,室内设计,"[视觉空间, 設計設計設計, 订制家具设计, 订制家居设计, 设备平面布局, 设置房屋装修,..."
3,软件测试,"[视觉仿真系统, 视频产品测试, 视频会议测试, 视频测试, 视频直播测试, 视频系统测试,..."
4,表演创作,"[视频演绎, 视频电视, 视频网红, 视频舞蹈, 视频表演, 角色关系, 角色创作能力, 角..."
...,...,...
195,务实性,"[觉得可以就行, 认真严谨钻研, 认真仔细勤快, 认真做事1, 认真做事2, 认真做事做好,..."
196,电商运营,"[视频电商运营, 设计电商运营, 课程电商运营, 负责京东天猫, 负责微商运营, 负责淘宝运..."
197,绘画技法,"[视觉插画设计, 视频文学漫画, 视频漫画, 角色原画制作, 角色原画培训, 角色原画手绘,..."
198,养发护发,"[认真对待美发, 设计儿童发型, 设计发型造型, 设计梳理发型, 设计精剪发型, 设计裁剪发..."


In [11]:
count_skill_words(d)

200
421962


In [12]:
len(d[d['1th_tag'] == '三维建模']['words'].values[0])

1899

### 载入JD

In [13]:
jds_before = []
for i in range(7):
    sheet_name = 'J' + str(i+1)
    print(sheet_name)
    jds_before.append(pd.read_excel(os.path.join(data_dir, version, '12月前.xlsx'), sheet_name='J' + str(i+1), engine='openpyxl'))
jds_before[-1]['APT_ID'] = 'J7'

J1


FileNotFoundError: [Errno 2] No such file or directory: 'data/games/12月前.xlsx'

In [ ]:
jds_before = pd.concat(jds_before)

In [ ]:
jds_before['december'] = 'before'

In [ ]:
jds_before

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,skills,position_name,degree,experience,low_salary,high_salary,december
0,9611.0,游戏策划,J1,235210397.0,游戏策划实习生,- 负责产品的文案创作，针对目标群体输出不同风格文案 - 参与团队协作，共同完成项目组内文...,游戏剧情策划#&#游戏关卡策划#&#玩法设计,游戏策划,本科,在校/应届,150.0,200.0,before
1,3469.0,游戏策划,J1,232190260.0,游戏策划师,- 岗位职责 - 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； - ...,游戏数值策划,游戏策划,本科,3-5年,20.0,40.0,before
2,1101.0,游戏策划,J1,230371418.0,游戏策划,- 岗位职责 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； 2、协调...,游戏数值策划,游戏策划,本科,1-3年,15.0,29.0,before
3,19784.0,文案编辑,J1,253607222.0,故事策划,- 岗位职责： 1.游戏图集故事策划，根据故事梗概，独立细化插画故事脚本输出。 ...,游戏剧情文案#&#休闲益智#&#文案策划#&#故事脚本#&#故事策划,文案编辑,本科,不限,3.0,5.0,before
4,4889.0,游戏策划,J1,235256426.0,游戏关卡策划,- 岗位职责： 1. 据主策的方向及要求，用已有元素进行关卡设计； 2.负责关卡体验及反...,玩法设计#&#游戏关卡策划#&#海外产品#&#手机游戏,游戏策划,本科,不限,8.0,13.0,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,1234.0,技术美术,J7,229932922.0,技术向技术美术-Pico,工作地点：北京、上海 职位描述 1、负责VR和移动端的虚拟世界社交产品内美术类需求的技术...,技术向技术美术,技术美术,本科,3-5年,30.0,60.0,before
1038,1235.0,技术美术,J7,229934156.0,技术向技术美术组长-Pico,职位描述 1、负责VR和移动端虚拟世界社交产品美术类需求的技术实现，解决美术人员日常遇到的...,技术向技术美术组长,技术美术,本科,5-10年,60.0,90.0,before
1039,1236.0,技术美术,J7,241319690.0,游戏技术美术ta,职位描述 1．制定资源规格，优化美术制作流程，确保资源符合引擎和项目规范 2．根据美术效果...,Unity#&#UE4#&#TA#&#OpenGL#&#Direct3D#&#PBR渲染原理,技术美术,本科,3-5年,15.0,30.0,before
1040,1237.0,技术美术,J7,244177421.0,unity地编,职位描述： 1．制定资源规格，优化美术制作流程，确保资源符合引擎和项目规范 2．根据美术效...,U3D#&#Shade#&#PBR渲染#&#TA#&#UE4#&#OpenGL/Direct3D,技术美术,本科,3-5年,15.0,30.0,before


In [ ]:
jds_after = []
for i in range(7):
    sheet_name = 'J' + str(i+1)
    print(sheet_name)
    jds_after.append(pd.read_excel(os.path.join(data_dir, version, '12月后.xlsx'), sheet_name='J' + str(i+1), engine='openpyxl'))
jds_after[-1]['APT_ID'] = 'J7'

J1
J2
J3
J4
J5
J6
J7


In [ ]:
jds_after = pd.concat(jds_after)

In [ ]:
jds_after['december'] = 'after'

In [ ]:
jds_after

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,position_name,skills,degree,experience,low_salary,high_salary,december
0,14906,游戏数值策划,J1,258865340,游戏主数值,岗位职责： 1、根据游戏市场及目标消费群体需求，以及项目规划，配合项目经理完成游戏经济体...,游戏数值策划,战斗系统设计#&#经济系统搭建#&#slg类型,大专,3-5年,20,35,after
1,222,游戏策划,J1,258422939,资深棋牌游戏策划,职责 1、对海外 slots、棋牌、休闲中某一类项目有较深的理解 2、从项目整体数值框架...,游戏策划,经济系统搭建#&#公式设计#&#平衡制定#&#休闲益智#&#模拟经营#&#卡牌/战棋,大专,5-10年,20,30,after
2,14122,游戏策划,J1,277035008,游戏策划,- 按主策划分配的工作任务，确保设计思路不偏离游戏框架，并达成预期目的,游戏策划,公式设计#&#平衡制定#&#技能系统设计#&#俯瞰#&#RTS游戏#&#模拟经营#&#卡牌/战棋,本科,1-3年,25,26,after
3,15126,游戏数值策划,J1,268841431,游戏数值策划-卡牌或mmorpg,岗位职责： 1、根据项目的类型和目标，参与数值框架的规划、设计与搭建； 2、负责为游戏内各...,游戏数值策划,怪物数值设计#&#经济系统搭建,本科,3-5年,18,28,after
4,7947,游戏策划,J1,261134408,游戏策划,- 根据市场、公司、游戏用户的需求设计合理、创新、有趣的游戏规则； - 根据游戏作的需求撰...,游戏策划,经济系统搭建,大专,1-3年,8,10,after
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,1399,技术美术,J7,277316690,资深角色原画设计师（GOG）,职位描述(JD)： 1.配合项目规划，完成针对商业化部分的角色设计。 2.跟随版本...,技术美术,美术,大专,5-10年,30,45,after
1395,1400,技术美术,J7,262855342,资深美术TA,岗位职责： 1. 使用Blender或者unity连连看工具实现主美对角色和场景的渲染风格化...,技术美术,Blender#&#渲染效果#&#渲染管线#&#性能优化#&#技术美术#&#TA,不限,5-10年,65,95,after
1396,1401,技术美术,J7,262869566,技术美术TA负责人,职责描述： 1、参与公司图形框架、管线的研发，配合技术美术提升项目渲染效果； 2、参与性能分...,技术美术,渲染#&#unity#&#图形#&#URP#&#技术美术#&#TA,大专,5-10年,40,60,after
1397,1402,技术美术,J7,262864761,资深动画TA,岗位职责： 1. 负责游戏项目中前沿动画效果的研究与落地。 2. 充分了解美术各环节开发中的...,技术美术,动画研究#&#动画表情系统#&#动画开发#&#动画控制器#&#动画工具开发,不限,5-10年,65,95,after


In [ ]:
jds = pd.concat([jds_before, jds_after])

In [ ]:
jds

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,skills,position_name,degree,experience,low_salary,high_salary,december
0,9611.0,游戏策划,J1,235210397.0,游戏策划实习生,- 负责产品的文案创作，针对目标群体输出不同风格文案 - 参与团队协作，共同完成项目组内文...,游戏剧情策划#&#游戏关卡策划#&#玩法设计,游戏策划,本科,在校/应届,150.0,200.0,before
1,3469.0,游戏策划,J1,232190260.0,游戏策划师,- 岗位职责 - 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； - ...,游戏数值策划,游戏策划,本科,3-5年,20.0,40.0,before
2,1101.0,游戏策划,J1,230371418.0,游戏策划,- 岗位职责 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； 2、协调...,游戏数值策划,游戏策划,本科,1-3年,15.0,29.0,before
3,19784.0,文案编辑,J1,253607222.0,故事策划,- 岗位职责： 1.游戏图集故事策划，根据故事梗概，独立细化插画故事脚本输出。 ...,游戏剧情文案#&#休闲益智#&#文案策划#&#故事脚本#&#故事策划,文案编辑,本科,不限,3.0,5.0,before
4,4889.0,游戏策划,J1,235256426.0,游戏关卡策划,- 岗位职责： 1. 据主策的方向及要求，用已有元素进行关卡设计； 2.负责关卡体验及反...,玩法设计#&#游戏关卡策划#&#海外产品#&#手机游戏,游戏策划,本科,不限,8.0,13.0,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,1399.0,技术美术,J7,277316690.0,资深角色原画设计师（GOG）,职位描述(JD)： 1.配合项目规划，完成针对商业化部分的角色设计。 2.跟随版本...,美术,技术美术,大专,5-10年,30.0,45.0,after
1395,1400.0,技术美术,J7,262855342.0,资深美术TA,岗位职责： 1. 使用Blender或者unity连连看工具实现主美对角色和场景的渲染风格化...,Blender#&#渲染效果#&#渲染管线#&#性能优化#&#技术美术#&#TA,技术美术,不限,5-10年,65.0,95.0,after
1396,1401.0,技术美术,J7,262869566.0,技术美术TA负责人,职责描述： 1、参与公司图形框架、管线的研发，配合技术美术提升项目渲染效果； 2、参与性能分...,渲染#&#unity#&#图形#&#URP#&#技术美术#&#TA,技术美术,大专,5-10年,40.0,60.0,after
1397,1402.0,技术美术,J7,262864761.0,资深动画TA,岗位职责： 1. 负责游戏项目中前沿动画效果的研究与落地。 2. 充分了解美术各环节开发中的...,动画研究#&#动画表情系统#&#动画开发#&#动画控制器#&#动画工具开发,技术美术,不限,5-10年,65.0,95.0,after


### 修正d - 过滤position_name

In [ ]:
position_names_org = list(set(jds['Position Name'].dropna(inplace=False).values))
print(len(position_names_org))
position_names = []
for name in position_names_org:
    if inc_chinese_char(name):
        position_names.append(name)
print(len(position_names))

83
71


In [ ]:
def position_words_filter(words):
    words_res = []
    for word in words:
        if word not in position_names:
            words_res.append(word)
    return words_res

In [ ]:
d['words'] = d['words'].progress_apply(position_words_filter)

100%|██████████| 200/200 [00:00<00:00, 499.03it/s]


In [ ]:
count_skill_words(d)

200
421911


### 修正d - stopwords

In [ ]:
# stopwords = ['熟练掌握', '岗位职责', '通话标准', '达能力强', '热爱教育行', '熟悉中小学', '相关算法', '相关法律法规', '通过网络', '节奏工作', '普通话流', '良好的客户服']
stopwords = ['熟练掌握', '岗位职责']

In [ ]:
def stop_words_filter(words):
    words_res = []
    for word in words:
        if word not in stopwords:
            words_res.append(word)
    return words_res

In [ ]:
d['words'] = d['words'].progress_apply(stop_words_filter)

100%|██████████| 200/200 [00:00<00:00, 3959.08it/s]


In [ ]:
count_skill_words(d)

200
421911


### 修正d - 添加6000技能词

In [14]:
d_v3_skills = pd.read_excel(os.path.join(skill_data_dir, '技能和性格词典_0320_2.xlsx'), engine='openpyxl', sheet_name='技能词')
d_v3_chars = pd.read_excel(os.path.join(skill_data_dir, '技能和性格词典_0320_2.xlsx'), engine='openpyxl', sheet_name='性格词')

In [15]:
d_tmp = {w: w for l in d['words'].values for w in l}

In [16]:
d_skills_new = []
for w in tqdm(d_v3_skills['skill_word'].values):
    if not d_tmp.get(w, None):
        d_skills_new.append(w)

d_chars_new = []
for w in tqdm(d_v3_chars['character_word'].values):
    if not d_tmp.get(w, None):
        d_chars_new.append(w)

print(len(d_skills_new))
print(len(d_chars_new))

100%|██████████| 1427/1427 [00:00<00:00, 1535078.69it/s]

2896
652


In [17]:
d = pd.concat([d, pd.DataFrame.from_dict({'1th_tag': '技能词补充', 'words': [d_skills_new]}), pd.DataFrame.from_dict({'1th_tag': '性格词补充', 'words': [d_chars_new]})]).reset_index()

In [18]:
count_skill_words(d)

202
425510


### 修正d - 保留length >= 4的词/非中文开头的词

In [19]:
def filter_words(words):
    return [word.lower() for word in words if len(word) >= 4 or not inc_chinese_char(word)]

In [20]:
d['words'] = d['words'].progress_apply(filter_words)

100%|██████████| 202/202 [00:00<00:00, 1979.18it/s]


In [21]:
print(d[d['1th_tag'] == '编程能力']['words'].values[0][-100: ])
print(d[d['1th_tag'] == '编程能力']['words'].values[0][: 100])

['vhdi语言', 'vhdl', 'vie架构', 'vip开发', 'vista', 'vis系统', 'vi体系执行', 'vi体系管理', 'vi系统规划', 'vi输出能力', 'vk', 'vlan原理', 'vlan技术', 'vlc', 'vlookup', 'vlsi设计', 'vmd制定', 'vmd设计', 'vmi流程', 'vmware虚拟化系统', 'vmware虚拟机', 'vmware数据中心', 'vmwaer虚拟化', 'vmware', 'vmware实施', 'vmware操作', 'vmware服务器', 'vmware管理', 'vmware系统', 'vm代理', 'vm基础知识', 'vm小助手', 'vm虚拟机', 'voc处理', 'vod系统', 'voip框架', 'volte', 'vos系统', 'vpn', 'vpn开发经验', 'vpn技术', 'vpn结构', 'vpn编程', 'vpn软硬件', 'vp和ep', 'vp系统', 'vrar开发', 'vray使用', 'vrue4u3d', 'vr游戏开发', 'vr程序开发', 'vr系统开发', 'vr系统的管理', 'vr软件开发', 'vs2005', 'vs2007', 'vs2008', 'vs201', 'vs2010以上', 'vs2010环境', 'vs2013', 'vs2015', 'vs20xx', 'vs99', 'vsave', 'vse慢动作', 'vslam方法', 'vsp', 'vspherekvm', 'vsply', 'vspy', 'vsto开发', 'vsto编程', 'vscode', 'vs工具', 'vs平台', 'vs开发工具', 'vs开发平台', 'vs开发环境', 'vs或qt', 'vs框架', 'vs软件开发', 'vs集成开发', 'vtk3d99', 'vtk开发经验', 'vtk开源软件', 'vtk等', 'vtm系统后台', 'vue', 'vueangular2', 'vuecss3', 'vuees6', 'vueh5', 'vue一年半', 'vue前端h5', '销售网络开发', '销售软件开发', '键盘程序', '长期从事it', '长链接服务器']
['规模化集

### 修正d - 变小写

In [22]:
def list_lower(words):
    res = []
    for word in words:
        res.append(word.lower())
    return res

In [23]:
d['words'] = d['words'].progress_apply(list_lower)

100%|██████████| 202/202 [00:00<00:00, 2011.67it/s]


### 修正d - 去重

In [24]:
def clear_repeat(words):
    return list(set(words))

In [25]:
d['words'] = d['words'].progress_apply(clear_repeat)

100%|██████████| 202/202 [00:00<00:00, 5113.34it/s]


In [26]:
count_skill_words(d)

202
421423


In [27]:
# 保存数据
d = d.drop('index', axis=1)
d.to_parquet(os.path.join(data_dir, version, 'd.parquet'))

OSError: Cannot save file into a non-existent directory: 'data/games'

### 数据统计

In [ ]:
jds

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,skills,position_name,degree,experience,low_salary,high_salary,december
0,9611.0,游戏策划,J1,235210397.0,游戏策划实习生,- 负责产品的文案创作，针对目标群体输出不同风格文案 - 参与团队协作，共同完成项目组内文...,游戏剧情策划#&#游戏关卡策划#&#玩法设计,游戏策划,本科,在校/应届,150.0,200.0,before
1,3469.0,游戏策划,J1,232190260.0,游戏策划师,- 岗位职责 - 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； - ...,游戏数值策划,游戏策划,本科,3-5年,20.0,40.0,before
2,1101.0,游戏策划,J1,230371418.0,游戏策划,- 岗位职责 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； 2、协调...,游戏数值策划,游戏策划,本科,1-3年,15.0,29.0,before
3,19784.0,文案编辑,J1,253607222.0,故事策划,- 岗位职责： 1.游戏图集故事策划，根据故事梗概，独立细化插画故事脚本输出。 ...,游戏剧情文案#&#休闲益智#&#文案策划#&#故事脚本#&#故事策划,文案编辑,本科,不限,3.0,5.0,before
4,4889.0,游戏策划,J1,235256426.0,游戏关卡策划,- 岗位职责： 1. 据主策的方向及要求，用已有元素进行关卡设计； 2.负责关卡体验及反...,玩法设计#&#游戏关卡策划#&#海外产品#&#手机游戏,游戏策划,本科,不限,8.0,13.0,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,1399.0,技术美术,J7,277316690.0,资深角色原画设计师（GOG）,职位描述(JD)： 1.配合项目规划，完成针对商业化部分的角色设计。 2.跟随版本...,美术,技术美术,大专,5-10年,30.0,45.0,after
1395,1400.0,技术美术,J7,262855342.0,资深美术TA,岗位职责： 1. 使用Blender或者unity连连看工具实现主美对角色和场景的渲染风格化...,Blender#&#渲染效果#&#渲染管线#&#性能优化#&#技术美术#&#TA,技术美术,不限,5-10年,65.0,95.0,after
1396,1401.0,技术美术,J7,262869566.0,技术美术TA负责人,职责描述： 1、参与公司图形框架、管线的研发，配合技术美术提升项目渲染效果； 2、参与性能分...,渲染#&#unity#&#图形#&#URP#&#技术美术#&#TA,技术美术,大专,5-10年,40.0,60.0,after
1397,1402.0,技术美术,J7,262864761.0,资深动画TA,岗位职责： 1. 负责游戏项目中前沿动画效果的研究与落地。 2. 充分了解美术各环节开发中的...,动画研究#&#动画表情系统#&#动画开发#&#动画控制器#&#动画工具开发,技术美术,不限,5-10年,65.0,95.0,after


In [ ]:
jds[jds['job_desc'].isna()]

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,skills,position_name,degree,experience,low_salary,high_salary,december
781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,before
20884,83941.0,NaN,J5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,before
15354,1153.0,网络客服,J6,237813828.0,游戏美妆种草APP~找打字客服~不想说话没事,NaN,不含销售的客服工作#&#办公室坐班#&#全职#&#售前客服#&#游戏美妆种草,网络客服,大专,不限,4.0,7.0,before
11958,19480.0,游戏角色,J4,273326926.0,角色原画（2D，欧美卡通）,NaN,2D角色原画#&#卡通类#&#欧美卡通,游戏角色,本科,1-3年,12.0,20.0,after


In [ ]:
jds_small = deepcopy(jds)

In [ ]:
jds_small = jds_small[jds_small['job_desc'].str.len() > 30]

### 分词 - 如果预存了d和jds_small则从这里开始

In [ ]:
d = pd.read_parquet(os.path.join(data_dir, version, 'd.parquet'))

In [ ]:
jds_small['job_desc'] = jds_small['job_desc'].fillna('')

In [ ]:
jds_small['job_desc'] = jds_small['job_desc'].progress_apply(lambda x: x.lower())

100%|██████████| 204511/204511 [00:00<00:00, 345926.29it/s]


In [ ]:
jds_small

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,skills,position_name,degree,experience,low_salary,high_salary,december
0,9611.0,游戏策划,J1,235210397.0,游戏策划实习生,- 负责产品的文案创作，针对目标群体输出不同风格文案 - 参与团队协作，共同完成项目组内文...,游戏剧情策划#&#游戏关卡策划#&#玩法设计,游戏策划,本科,在校/应届,150.0,200.0,before
1,3469.0,游戏策划,J1,232190260.0,游戏策划师,- 岗位职责 - 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； - ...,游戏数值策划,游戏策划,本科,3-5年,20.0,40.0,before
2,1101.0,游戏策划,J1,230371418.0,游戏策划,- 岗位职责 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； 2、协调...,游戏数值策划,游戏策划,本科,1-3年,15.0,29.0,before
3,19784.0,文案编辑,J1,253607222.0,故事策划,- 岗位职责： 1.游戏图集故事策划，根据故事梗概，独立细化插画故事脚本输出。 ...,游戏剧情文案#&#休闲益智#&#文案策划#&#故事脚本#&#故事策划,文案编辑,本科,不限,3.0,5.0,before
4,4889.0,游戏策划,J1,235256426.0,游戏关卡策划,- 岗位职责： 1. 据主策的方向及要求，用已有元素进行关卡设计； 2.负责关卡体验及反...,玩法设计#&#游戏关卡策划#&#海外产品#&#手机游戏,游戏策划,本科,不限,8.0,13.0,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,1399.0,技术美术,J7,277316690.0,资深角色原画设计师（GOG）,职位描述(jd)： 1.配合项目规划，完成针对商业化部分的角色设计。 2.跟随版本...,美术,技术美术,大专,5-10年,30.0,45.0,after
1395,1400.0,技术美术,J7,262855342.0,资深美术TA,岗位职责： 1. 使用blender或者unity连连看工具实现主美对角色和场景的渲染风格化...,Blender#&#渲染效果#&#渲染管线#&#性能优化#&#技术美术#&#TA,技术美术,不限,5-10年,65.0,95.0,after
1396,1401.0,技术美术,J7,262869566.0,技术美术TA负责人,职责描述： 1、参与公司图形框架、管线的研发，配合技术美术提升项目渲染效果； 2、参与性能分...,渲染#&#unity#&#图形#&#URP#&#技术美术#&#TA,技术美术,大专,5-10年,40.0,60.0,after
1397,1402.0,技术美术,J7,262864761.0,资深动画TA,岗位职责： 1. 负责游戏项目中前沿动画效果的研究与落地。 2. 充分了解美术各环节开发中的...,动画研究#&#动画表情系统#&#动画开发#&#动画控制器#&#动画工具开发,技术美术,不限,5-10年,65.0,95.0,after


In [ ]:
d_words = list(set(itertools.chain.from_iterable(d.words.values)))

In [ ]:
# 测试一下有没有一个词对到两个类
d_inverse = defaultdict(list)
for idx, row in d.iterrows():
    for w in row['words']:
        d_inverse[w].append(row['1th_tag'])
d_inverse = dict(d_inverse)
for k, v in d_inverse.items():
    if len(v) > 1:
        print(k)
        break

debug


In [ ]:
d_inverse['web原型设计']

['原型设计', '技能词补充']

In [ ]:
len(d_words)

421063

In [ ]:
for word in tqdm(d_words):
    if len(word) > 1:
        jieba.add_word(word)

100%|██████████| 421063/421063 [00:08<00:00, 47724.81it/s]


In [ ]:
pandarallel.initialize(nb_workers=20, use_memory_fs=False, progress_bar=True)

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [ ]:
t1 = time.time()
jds_small['tokens'] = jds_small['job_desc'].parallel_apply(jieba_jd_lcut)
t2 = time.time()

In [ ]:
(t2 - t1) / 60

0.29028406540552776

In [ ]:
jds_small

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,skills,position_name,degree,experience,low_salary,high_salary,december,tokens
0,9611.0,游戏策划,J1,235210397.0,游戏策划实习生,- 负责产品的文案创作，针对目标群体输出不同风格文案 - 参与团队协作，共同完成项目组内文...,游戏剧情策划#&#游戏关卡策划#&#玩法设计,游戏策划,本科,在校/应届,150.0,200.0,before,"[-, 负责, 产品, 的, 文案创作, 针对, 目标群体, 输出, 不同, 风格, 文案,..."
1,3469.0,游戏策划,J1,232190260.0,游戏策划师,- 岗位职责 - 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； - ...,游戏数值策划,游戏策划,本科,3-5年,20.0,40.0,before,"[-, 岗位职责, -, 1, 、, 配合, 产品定位, 完成, 游戏, 中, 玩法, 、,..."
2,1101.0,游戏策划,J1,230371418.0,游戏策划,- 岗位职责 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； 2、协调...,游戏数值策划,游戏策划,本科,1-3年,15.0,29.0,before,"[-, 岗位职责, 1, 、, 配合, 产品定位, 完成, 游戏, 中, 玩法, 、, 数值..."
3,19784.0,文案编辑,J1,253607222.0,故事策划,- 岗位职责： 1.游戏图集故事策划，根据故事梗概，独立细化插画故事脚本输出。 ...,游戏剧情文案#&#休闲益智#&#文案策划#&#故事脚本#&#故事策划,文案编辑,本科,不限,3.0,5.0,before,"[-, 岗位职责, ：, 1, 游戏, 图集, 故事策划, 根据, 故事梗概, 独立, 细化..."
4,4889.0,游戏策划,J1,235256426.0,游戏关卡策划,- 岗位职责： 1. 据主策的方向及要求，用已有元素进行关卡设计； 2.负责关卡体验及反...,玩法设计#&#游戏关卡策划#&#海外产品#&#手机游戏,游戏策划,本科,不限,8.0,13.0,before,"[-, 岗位职责, ：, 1, 据主策, 的, 方向, 及, 要求, 用, 已有, 元素, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,1399.0,技术美术,J7,277316690.0,资深角色原画设计师（GOG）,职位描述(jd)： 1.配合项目规划，完成针对商业化部分的角色设计。 2.跟随版本...,美术,技术美术,大专,5-10年,30.0,45.0,after,"[职位, 描述, (, jd, ), ：, 1, 配合, 项目规划, 完成, 针对, 商业化..."
1395,1400.0,技术美术,J7,262855342.0,资深美术TA,岗位职责： 1. 使用blender或者unity连连看工具实现主美对角色和场景的渲染风格化...,Blender#&#渲染效果#&#渲染管线#&#性能优化#&#技术美术#&#TA,技术美术,不限,5-10年,65.0,95.0,after,"[岗位职责, ：, 1, 使用, blender, 或者, unity, 连连看, 工具, ..."
1396,1401.0,技术美术,J7,262869566.0,技术美术TA负责人,职责描述： 1、参与公司图形框架、管线的研发，配合技术美术提升项目渲染效果； 2、参与性能分...,渲染#&#unity#&#图形#&#URP#&#技术美术#&#TA,技术美术,大专,5-10年,40.0,60.0,after,"[职责, 描述, ：, 1, 、, 参与, 公司, 图形, 框架, 、, 管线, 的, 研发..."
1397,1402.0,技术美术,J7,262864761.0,资深动画TA,岗位职责： 1. 负责游戏项目中前沿动画效果的研究与落地。 2. 充分了解美术各环节开发中的...,动画研究#&#动画表情系统#&#动画开发#&#动画控制器#&#动画工具开发,技术美术,不限,5-10年,65.0,95.0,after,"[岗位职责, ：, 1, 负责, 游戏, 项目, 中, 前沿, 动画效果, 的, 研究, 与..."


In [ ]:
jds_small.to_parquet(os.path.join(data_dir, version, 'jds_small_split.parquet'))

In [ ]:
count_skill_words(d)

202
421377


In [ ]:
d_dict = d.set_index('1th_tag').to_dict()['words']

In [ ]:
d_dict['编程能力']

array(['云计算系统架构', 'apache服务器', '自定义注解', ..., 'arm框架', 'mcn行业背景', 'rpc服务'],
      dtype=object)

In [ ]:
def match_skill(tokens):
    res = defaultdict(list)
    for token in tokens:
        tags = d_inverse.get(token, None)
        if tags:
            for tag in tags:
                res[tag].append(token)
    return dict(res)

In [ ]:
jds_small['skill_list'] = jds_small['tokens'].progress_apply(match_skill)

100%|██████████| 204511/204511 [00:11<00:00, 17714.40it/s]


In [ ]:
jds_small

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,skills,position_name,degree,experience,low_salary,high_salary,december,tokens,skill_list
0,9611.0,游戏策划,J1,235210397.0,游戏策划实习生,- 负责产品的文案创作，针对目标群体输出不同风格文案 - 参与团队协作，共同完成项目组内文...,游戏剧情策划#&#游戏关卡策划#&#玩法设计,游戏策划,本科,在校/应届,150.0,200.0,before,"[-, 负责, 产品, 的, 文案创作, 针对, 目标群体, 输出, 不同, 风格, 文案,...","{'写作能力': ['文案创作', '文字素养'], '团队合作': ['团队协作', '擅..."
1,3469.0,游戏策划,J1,232190260.0,游戏策划师,- 岗位职责 - 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； - ...,游戏数值策划,游戏策划,本科,3-5年,20.0,40.0,before,"[-, 岗位职责, -, 1, 、, 配合, 产品定位, 完成, 游戏, 中, 玩法, 、,...","{'问题解决': ['产品定位'], '编程能力': ['系统设计', 'cocos', '..."
2,1101.0,游戏策划,J1,230371418.0,游戏策划,- 岗位职责 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； 2、协调...,游戏数值策划,游戏策划,本科,1-3年,15.0,29.0,before,"[-, 岗位职责, 1, 、, 配合, 产品定位, 完成, 游戏, 中, 玩法, 、, 数值...","{'问题解决': ['产品定位'], '编程能力': ['系统设计', 'cocos', '..."
3,19784.0,文案编辑,J1,253607222.0,故事策划,- 岗位职责： 1.游戏图集故事策划，根据故事梗概，独立细化插画故事脚本输出。 ...,游戏剧情文案#&#休闲益智#&#文案策划#&#故事脚本#&#故事策划,文案编辑,本科,不限,3.0,5.0,before,"[-, 岗位职责, ：, 1, 游戏, 图集, 故事策划, 根据, 故事梗概, 独立, 细化...","{'策划制作': ['故事策划', '故事脚本'], '问题解决': ['提出建议'], '..."
4,4889.0,游戏策划,J1,235256426.0,游戏关卡策划,- 岗位职责： 1. 据主策的方向及要求，用已有元素进行关卡设计； 2.负责关卡体验及反...,玩法设计#&#游戏关卡策划#&#海外产品#&#手机游戏,游戏策划,本科,不限,8.0,13.0,before,"[-, 岗位职责, ：, 1, 据主策, 的, 方向, 及, 要求, 用, 已有, 元素, ...","{'展示陈列': ['关卡设计'], '资源拓展': ['用户数据', '日活跃用户'], ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,1399.0,技术美术,J7,277316690.0,资深角色原画设计师（GOG）,职位描述(jd)： 1.配合项目规划，完成针对商业化部分的角色设计。 2.跟随版本...,美术,技术美术,大专,5-10年,30.0,45.0,after,"[职位, 描述, (, jd, ), ：, 1, 配合, 项目规划, 完成, 针对, 商业化...","{'统筹规划': ['项目规划'], '动态设计': ['角色设计', '角色设计', '角..."
1395,1400.0,技术美术,J7,262855342.0,资深美术TA,岗位职责： 1. 使用blender或者unity连连看工具实现主美对角色和场景的渲染风格化...,Blender#&#渲染效果#&#渲染管线#&#性能优化#&#技术美术#&#TA,技术美术,不限,5-10年,65.0,95.0,after,"[岗位职责, ：, 1, 使用, blender, 或者, unity, 连连看, 工具, ...","{'编程能力': ['unity', 'pbr', 'unity', '性能优化', '编程..."
1396,1401.0,技术美术,J7,262869566.0,技术美术TA负责人,职责描述： 1、参与公司图形框架、管线的研发，配合技术美术提升项目渲染效果； 2、参与性能分...,渲染#&#unity#&#图形#&#URP#&#技术美术#&#TA,技术美术,大专,5-10年,40.0,60.0,after,"[职责, 描述, ：, 1, 、, 参与, 公司, 图形, 框架, 、, 管线, 的, 研发...","{'场景渲染': ['渲染效果', '图形渲染', '渲染流程', 'pbr渲染'], '系..."
1397,1402.0,技术美术,J7,262864761.0,资深动画TA,岗位职责： 1. 负责游戏项目中前沿动画效果的研究与落地。 2. 充分了解美术各环节开发中的...,动画研究#&#动画表情系统#&#动画开发#&#动画控制器#&#动画工具开发,技术美术,不限,5-10年,65.0,95.0,after,"[岗位职责, ：, 1, 负责, 游戏, 项目, 中, 前沿, 动画效果, 的, 研究, 与...","{'动态设计': ['动画效果', '了解动画', '动画效果'], '艺术审美': ['美..."


In [ ]:
jds_small = jds_small.drop('tokens', axis=1)

In [ ]:
for december in ['before', 'after']:
    with pd.ExcelWriter(os.path.join(out_dir, version, december + '.xlsx')) as writer:
        for i in tqdm(range(7)):
            apt_id = 'J' + str(i+1)
            jds_small[(jds_small['december'] == december) & (jds_small['APT_ID'] == apt_id)].drop('december', axis=1).to_excel(writer, sheet_name=apt_id, index=False)

100%|██████████| 7/7 [00:11<00:00,  1.62s/it]


### sklearn

In [ ]:
jds_small['tokens'] = jds_small['tokens'].progress_apply(lambda x: ' '.join(x))

In [ ]:
jds_small

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,skills,position_name,degree,experience,low_salary,high_salary,december,tokens
0,9611.0,游戏策划,J1,235210397.0,游戏策划实习生,- 负责产品的文案创作，针对目标群体输出不同风格文案 - 参与团队协作，共同完成项目组内文...,游戏剧情策划#&#游戏关卡策划#&#玩法设计,游戏策划,本科,在校/应届,150.0,200.0,before,- 负责 产品 的 文案创作 针对 目标群体 输出 不同 风格 文案 - 参与 团队协作 共...
1,3469.0,游戏策划,J1,232190260.0,游戏策划师,- 岗位职责 - 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； - ...,游戏数值策划,游戏策划,本科,3-5年,20.0,40.0,before,- 岗位职责 - 1 、 配合 产品定位 完成 游戏 中 玩法 、 数值 及其 他 密切相关...
2,1101.0,游戏策划,J1,230371418.0,游戏策划,- 岗位职责 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； 2、协调...,游戏数值策划,游戏策划,本科,1-3年,15.0,29.0,before,- 岗位职责 1 、 配合 产品定位 完成 游戏 中 玩法 、 数值 及其 他 密切相关 的...
3,19784.0,文案编辑,J1,253607222.0,故事策划,- 岗位职责： 1.游戏图集故事策划，根据故事梗概，独立细化插画故事脚本输出。 ...,游戏剧情文案#&#休闲益智#&#文案策划#&#故事脚本#&#故事策划,文案编辑,本科,不限,3.0,5.0,before,- 岗位职责 ： 1 游戏 图集 故事策划 根据 故事梗概 独立 细化 插画 故事脚本 输出...
4,4889.0,游戏策划,J1,235256426.0,游戏关卡策划,- 岗位职责： 1. 据主策的方向及要求，用已有元素进行关卡设计； 2.负责关卡体验及反...,玩法设计#&#游戏关卡策划#&#海外产品#&#手机游戏,游戏策划,本科,不限,8.0,13.0,before,- 岗位职责 ： 1 据主策 的 方向 及 要求 用 已有 元素 进行 关卡设计 ； 2 负...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,1399.0,技术美术,J7,277316690.0,资深角色原画设计师（GOG）,职位描述(jd)： 1.配合项目规划，完成针对商业化部分的角色设计。 2.跟随版本...,美术,技术美术,大专,5-10年,30.0,45.0,after,职位 描述 ( jd ) ： 1 配合 项目规划 完成 针对 商业化 部分 的 角色设计 2...
1395,1400.0,技术美术,J7,262855342.0,资深美术TA,岗位职责： 1. 使用blender或者unity连连看工具实现主美对角色和场景的渲染风格化...,Blender#&#渲染效果#&#渲染管线#&#性能优化#&#技术美术#&#TA,技术美术,不限,5-10年,65.0,95.0,after,岗位职责 ： 1 使用 blender 或者 unity 连连看 工具 实现 主美 对 角色...
1396,1401.0,技术美术,J7,262869566.0,技术美术TA负责人,职责描述： 1、参与公司图形框架、管线的研发，配合技术美术提升项目渲染效果； 2、参与性能分...,渲染#&#unity#&#图形#&#URP#&#技术美术#&#TA,技术美术,大专,5-10年,40.0,60.0,after,职责 描述 ： 1 、 参与 公司 图形 框架 、 管线 的 研发 配合 技术 美术 提升 ...
1397,1402.0,技术美术,J7,262864761.0,资深动画TA,岗位职责： 1. 负责游戏项目中前沿动画效果的研究与落地。 2. 充分了解美术各环节开发中的...,动画研究#&#动画表情系统#&#动画开发#&#动画控制器#&#动画工具开发,技术美术,不限,5-10年,65.0,95.0,after,岗位职责 ： 1 负责 游戏 项目 中 前沿 动画效果 的 研究 与 落地 2 充分 了解 ...


In [ ]:
jds_small.to_parquet(os.path.join(data_dir, version, 'jds_small_split.parquet'))

#### 计算词频

In [ ]:
jds_small = pd.read_parquet(os.path.join(data_dir, version, 'jds_small_split.parquet'))

In [ ]:
jds_small

,Unnamed: 0,Position Name,APT_ID,job_id,job_title,job_desc,skills,position_name,degree,experience,low_salary,high_salary,december,tokens
0,9611.0,游戏策划,J1,235210397.0,游戏策划实习生,- 负责产品的文案创作，针对目标群体输出不同风格文案 - 参与团队协作，共同完成项目组内文...,游戏剧情策划#&#游戏关卡策划#&#玩法设计,游戏策划,本科,在校/应届,150.0,200.0,before,- 负责 产品 的 文案创作 针对 目标群体 输出 不同 风格 文案 - 参与 团队协作 共...
1,3469.0,游戏策划,J1,232190260.0,游戏策划师,- 岗位职责 - 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； - ...,游戏数值策划,游戏策划,本科,3-5年,20.0,40.0,before,- 岗位职责 - 1 、 配合 产品定位 完成 游戏 中 玩法 、 数值 及其 他 密切相关...
2,1101.0,游戏策划,J1,230371418.0,游戏策划,- 岗位职责 1、配合产品定位，完成游戏中玩法、数值及其他密切相关的系统设计工作； 2、协调...,游戏数值策划,游戏策划,本科,1-3年,15.0,29.0,before,- 岗位职责 1 、 配合 产品定位 完成 游戏 中 玩法 、 数值 及其 他 密切相关 的...
3,19784.0,文案编辑,J1,253607222.0,故事策划,- 岗位职责： 1.游戏图集故事策划，根据故事梗概，独立细化插画故事脚本输出。 ...,游戏剧情文案#&#休闲益智#&#文案策划#&#故事脚本#&#故事策划,文案编辑,本科,不限,3.0,5.0,before,- 岗位职责 ： 1 游戏 图集 故事策划 根据 故事梗概 独立 细化 插画 故事脚本 输出...
4,4889.0,游戏策划,J1,235256426.0,游戏关卡策划,- 岗位职责： 1. 据主策的方向及要求，用已有元素进行关卡设计； 2.负责关卡体验及反...,玩法设计#&#游戏关卡策划#&#海外产品#&#手机游戏,游戏策划,本科,不限,8.0,13.0,before,- 岗位职责 ： 1 据主策 的 方向 及 要求 用 已有 元素 进行 关卡设计 ； 2 负...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,1399.0,技术美术,J7,277316690.0,资深角色原画设计师（GOG）,职位描述(jd)： 1.配合项目规划，完成针对商业化部分的角色设计。 2.跟随版本...,美术,技术美术,大专,5-10年,30.0,45.0,after,职位 描述 ( jd ) ： 1 配合 项目规划 完成 针对 商业化 部分 的 角色设计 2...
1395,1400.0,技术美术,J7,262855342.0,资深美术TA,岗位职责： 1. 使用blender或者unity连连看工具实现主美对角色和场景的渲染风格化...,Blender#&#渲染效果#&#渲染管线#&#性能优化#&#技术美术#&#TA,技术美术,不限,5-10年,65.0,95.0,after,岗位职责 ： 1 使用 blender 或者 unity 连连看 工具 实现 主美 对 角色...
1396,1401.0,技术美术,J7,262869566.0,技术美术TA负责人,职责描述： 1、参与公司图形框架、管线的研发，配合技术美术提升项目渲染效果； 2、参与性能分...,渲染#&#unity#&#图形#&#URP#&#技术美术#&#TA,技术美术,大专,5-10年,40.0,60.0,after,职责 描述 ： 1 、 参与 公司 图形 框架 、 管线 的 研发 配合 技术 美术 提升 ...
1397,1402.0,技术美术,J7,262864761.0,资深动画TA,岗位职责： 1. 负责游戏项目中前沿动画效果的研究与落地。 2. 充分了解美术各环节开发中的...,动画研究#&#动画表情系统#&#动画开发#&#动画控制器#&#动画工具开发,技术美术,不限,5-10年,65.0,95.0,after,岗位职责 ： 1 负责 游戏 项目 中 前沿 动画效果 的 研究 与 落地 2 充分 了解 ...


In [ ]:
cv = CountVectorizer(token_pattern=r'(?u)\b\w+\b', stop_words=stopwords)

In [ ]:
job_small_doc = jds_small['tokens'].values.tolist()

In [ ]:
t1 = time.time()
cv_fit = cv.fit_transform(job_small_doc)
dictionary = cv.get_feature_names()
t2 = time.time()

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', 'β', 'δ', 'λ', 'ξ', 'ψ', 'в', 'ⅲ', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '元', '吨', '数', '日', '末', '０', '１', '１２', '２', '３', '５', 'ａ', 'ｂ', 'ｃ', 'ｅ', 'ｆ', 'ｌ', 'ｌｉ', 'ｒ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWa

In [ ]:
(t2 - t1) / 60

0.2702498515446981

In [ ]:
len(dictionary)

129759

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
dictionary_dict = {w: i for i, w in enumerate(tqdm(dictionary))}

In [ ]:
dictionary.index('000')

1

In [ ]:
d_words_valid = [word for word in tqdm(d_words) if dictionary_dict.get(word, None)]
indices = [dictionary_dict[word] for word in tqdm(d_words_valid)]

In [ ]:
### 抽样结果中最终有效的词
len(d_words_valid)

61374

#### 数据切片，仅保留技能词所在的列

In [ ]:
cv_count = cv_fit[:, indices].toarray()
cv_count.shape

(204511, 61374)

In [ ]:
df = pd.DataFrame(cv_count)
df.columns = d_words_valid

In [ ]:
# 测试
print(jds_small.tokens.values[0])
print('==========')
for k, v in zip(df.columns, df.values[0]):
    if v != 0:
        print(k + str(v))

- 负责 产品 的 文案创作 针对 目标群体 输出 不同 风格 文案 - 参与 团队协作 共同完成 项目组 内 文案 方面 的 沟通 、 协调 、 创作 等 工作 职位 要求 - base 上海 23 年 及 以后 毕业 大学本科 或 研究生 在读 中文 、 新闻 、 广告 等 相关 专业 优先 - 具有 一定 的 逻辑思维能力 和 基本 的 文字素养 - 有 游戏行业经验 优先 - 对 市场 把握 敏感 擅长团队协作 具有 理解和沟通 能力 职位 亮点 - 孵化 新 产品 的 经历 做 面向未来 的 探索 - 足够 广阔 的 发挥 空间 扁平化 的 管理 - 简单 的 人际关系 专注 且 靠 谱 的 同事 - 双休 、 员工福利 、 高 潜力 的 期权 增长 空间
文字素养1
团队协作1
文案创作1
游戏行业经验1
理解和沟通1
擅长团队协作1
逻辑思维能力1


In [ ]:
df

,普通话一般,居住空间,管理微信公众,能快速调整,维护调试,伙伴关系,冰雪世界,结构维护,资深工业设计,广告账户管理,...,独立完成业务,操作系统linux,无严重口音,熟练掌握ppt,信息安全规范,定位潜在客户,halcon,移动架构设计,c软件开发,精准销售
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
204507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
204508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
204509,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.to_parquet(os.path.join(data_dir, version, 'df_skills.parquet'))

### 整理结果

In [ ]:
# 保存数据
df.to_parquet(os.path.join(data_dir, version, 'cv_count.parquet'))